# Censorship Detection with Machine Learning

This notebook applies machine learning models to the preprocessed ASN connectivity data to detect censorship events.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import warnings
warnings.filterwarnings('ignore')

# For plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Load Preprocessed Data

Let's load the data we preprocessed in the previous notebook.

In [ ]:
# Load the preprocessed data
df = pd.read_csv('../data/processed/preprocessed_for_ml.csv')

print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.shape[1]}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Countries: {df['country'].nunique()}")

# Convert date column back to datetime
df['date'] = pd.to_datetime(df['date'])

print(f"\nSample of data:")
print(df.head(3))

## Data Quality Check and Target Analysis

Let's examine the target variables and data quality.

In [ ]:
# Check for potential target variables
target_cols = [col for col in df.columns if 'censorship_indicator' in col]
print(f"Potential target columns: {target_cols}")

# Analyze target distributions
for target_col in target_cols:
    if target_col in df.columns:
        target_counts = df[target_col].value_counts()
        print(f"\n{target_col} distribution:")
        print(target_counts)
        print(f"Positive ratio: {df[target_col].mean():.6f}")

In [ ]:
# If no censorship indicators were found, create based on significant drops in connectivity
if not any(col for col in df.columns if 'censorship_indicator' in col):
    print("Creating target based on significant drops in foreign connectivity...")
    
    # If we have foreign_neighbours_share, create target based on significant drops
    if 'foreign_neighbours_share' in df.columns:
        # Calculate country-specific rolling baseline
        df['foreign_conn_baseline'] = df.groupby('country')['foreign_neighbours_share'].transform(
            lambda x: x.rolling(window=30, min_periods=7).median()
        )
        
        # Create binary target for significant drops
        df['censorship_target'] = (
            (df['foreign_neighbours_share'] < df['foreign_conn_baseline'] * 0.5) &
            (df['foreign_neighbours_share'].notna()) &
            (df['foreign_conn_baseline'].notna())
        ).astype(int)
        
        target_col = 'censorship_target'
    else:
        # If we have ASN count, create target based on significant drops
        if 'asn_count' in df.columns:
            df['asn_count_baseline'] = df.groupby('country')['asn_count'].transform(
                lambda x: x.rolling(window=30, min_periods=7).median()
            )
            
            df['censorship_target'] = (
                (df['asn_count'] < df['asn_count_baseline'] * 0.5) &
                (df['asn_count'].notna()) &
                (df['asn_count_baseline'].notna())
            ).astype(int)
            
            target_col = 'censorship_target'
        else:
            # Fallback: use any numeric column with significant changes
            numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
            # Pick a column that has reasonable variance
            for col in numeric_cols:
                if df[col].std() > 0 and df[col].nunique() > 10:
                    df['censorship_target'] = df[col]
                    # Normalize and create binary classification problem
                    median_val = df['censorship_target'].median()
                    df['censorship_target'] = (df['censorship_target'] < median_val).astype(int)
                    target_col = 'censorship_target'
                    break
        
    print(f"Created target variable {target_col} with {df[target_col].sum()} positive cases ({df[target_col].mean():.4f} ratio)")
else:
    # Use existing target
    for col in df.columns:
        if 'censorship_indicator' in col and df[col].sum() > 0:
            target_col = col
            break

## Feature Selection

Select appropriate features for the model, excluding date, country, and target columns.

In [ ]:
# Define features to exclude
exclude_cols = ['date', 'country', target_col]  # Exclude target column

# Add any other columns that shouldn't be features
exclude_cols.extend([col for col in df.columns if 'baseline' in col or 'rolling_median' in col])

# Select only numeric features
feature_cols = [col for col in df.columns 
                if col not in exclude_cols 
                and df[col].dtype in ['int64', 'float64']]

print(f"Number of features selected: {len(feature_cols)}")
print(f"Features: {feature_cols[:15]}... ({'truncated' if len(feature_cols) > 15 else 'complete'})")

# Prepare feature matrix X and target vector y
X = df[feature_cols].copy()
y = df[target_col].copy()

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Positive target ratio: {y.mean():.6f}")

## Handle Class Imbalance

Censorship events are typically rare, so we'll need to handle class imbalance.

In [ ]:
# Calculate class weights to handle imbalance
classes = np.unique(y)
class_weights = compute_class_weight('balanced', classes=classes, y=y)
class_weight_dict = dict(zip(classes, class_weights))

print(f"Class distribution:")
for cls in classes:
    count = (y == cls).sum()
    print(f"  Class {cls}: {count} samples ({count/len(y)*100:.2f}%)")

print(f"\nClass weights: {class_weight_dict}")

## Split Data

Split the data for training and testing. We'll use a time-based split to avoid data leakage.

In [ ]:
# Add date back to X for time-based splitting
X_with_date = X.copy()
X_with_date['date'] = df['date']

# Sort by date to ensure chronological order
X_sorted = X_with_date.sort_values('date')
y_sorted = y.loc[X_sorted.index]

# Remove date column from features before modeling
X_final = X_sorted.drop('date', axis=1)

# Use the last 20% for testing (time-based split)
split_idx = int(len(X_final) * 0.8)

X_train = X_final.iloc[:split_idx]
X_test = X_final.iloc[split_idx:]
y_train = y_sorted.iloc[:split_idx]
y_test = y_sorted.iloc[split_idx:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Positive cases in training: {y_train.sum()} ({y_train.mean():.4f} ratio)")
print(f"Positive cases in test: {y_test.sum()} ({y_test.mean():.4f} ratio)")

## Model Training

Train multiple models to compare performance.

In [ ]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(
        random_state=42, 
        class_weight=class_weight_dict,
        max_iter=1000
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        class_weight=class_weight_dict,
        n_jobs=-1,
        min_samples_split=10,
        min_samples_leaf=5
    )
}

# Initialize scaler for models that need it
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train and evaluate models
results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Use scaled features for Logistic Regression, original for Random Forest
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # Store results
    results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"{name} Metrics:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  AUC: {auc:.4f}")
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"  Confusion Matrix:")
    print(f"    {cm}")

## Model Evaluation and Results

Analyze the results and generate detailed reports.

In [ ]:
# Detailed classification reports
for name in models.keys():
    print(f"\n{name} Classification Report:")
    print(classification_report(y_test, results[name]['y_pred'], zero_division=0))

In [ ]:
# Plot ROC curves
plt.figure(figsize=(10, 8))

from sklearn.metrics import roc_curve

for name in models.keys():
    fpr, tpr, _ = roc_curve(y_test, results[name]['y_pred_proba'])
    auc = results[name]['auc']
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Censorship Detection Models')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Feature importance for Random Forest
rf_model = models['Random Forest']

# Get feature importances
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': X_final.columns,
    'importance': importances
}).sort_values('importance', ascending=False)

print("Top 15 Most Important Features:")
print(feature_importance_df.head(15))

# Plot feature importances
plt.figure(figsize=(12, 8))
top_features = feature_importance_df.head(15)
sns.barplot(data=top_features, y='feature', x='importance')
plt.title('Top 15 Feature Importances (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## Model Performance Summary

Summarize the findings.

In [ ]:
print("Model Performance Summary:")
print("="*60)

summary_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[name]['accuracy'] for name in results.keys()],
    'Precision': [results[name]['precision'] for name in results.keys()],
    'Recall': [results[name]['recall'] for name in results.keys()],
    'F1-Score': [results[name]['f1'] for name in results.keys()],
    'AUC': [results[name]['auc'] for name in results.keys()]
})

print(summary_df.to_string(index=False))

# Identify best model
best_model_name = summary_df.loc[summary_df['F1-Score'].idxmax(), 'Model']
print(f"\nBest Model: {best_model_name} (F1-Score: {summary_df['F1-Score'].max():.4f})")

## Conclusion and Next Steps

Summarize results and suggest improvements.

In [ ]:
print("CENSORSHIP DETECTION MODELING RESULTS")
print("=" * 50)
print(f"Dataset shape: {df.shape}")
print(f"Features used: {len(feature_cols)}")
print(f"Target variable: {target_col}")
print(f"Positive cases: {y.sum()} ({y.mean():.4f} ratio)")
print(f"Time range: {df['date'].min()} to {df['date'].max()}")
print(f"Countries: {df['country'].nunique()}")
print()

for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  - Accuracy: {metrics['accuracy']:.4f}")
    print(f"  - Precision: {metrics['precision']:.4f}")
    print(f"  - Recall: {metrics['recall']:.4f}")
    print(f"  - F1-Score: {metrics['f1']:.4f}")
    print(f"  - AUC: {metrics['auc']:.4f}")
    print()